<h2>0. Introduction </h2>
The Subway Challenge is a challenge in which participants must navigate the entire New York City Subway system in the shortest time possible. This ride is also known as the Rapid Transit Challenge and the Ultimate Ride. Although the challenge requires competitors to stop at all 472 stations, no person currently holds that record <a href="https://en.wikipedia.org/wiki/Subway_Challenge#Guinness_Record_times"> [] </a>. The most recent record of 21H:28M:14S was set on July 22, 2016 by Matthew Ahn for the 469-Station Challenge <a href="https://www.timeout.com/newyork/blog/solo-straphanger-sets-new-all-station-subway-world-record-090616"> []</a>. Other than beginning at the Rockaway Park-Beach 116th Street Station and ending at Flushing-Main Street nobody knows the method he used to beat the record. The subway system can be modeled as a graph where the stations are the nodes, connected by the train tracks which are the edges. <br></br>

<p style='text-align:center'> <b>The goal of this project is to use graph theory to determine a set of paths that could potentially be used to beat the current record.</b> </p>



<h2>The Data Engineering </h2>
To solve this problem a graph representation of the subway system needs to be constructed. The subway system can be modeled as a weighted undirected graph, where the weights on the edges are the time it takes to get from one station to the next. Since you can travel in both directions on each line the direction is not needed <i>(there is one station that is an exception)</i>. The actual map of the system needs to be translated into nodes and edges; to simplify this translation the <a href="https://new.mta.info/map/5336">Late Night Subway Service</a> map is used. In the late night subway map all stations are served though not all lines run; most lines run local, making all stops. The late night map is a starting point to attempt beating the challenge. The map cannot be used, as is, to beat the challenge because the map is only valid from 00:00 - 06:00 everyday. The late night map tells you <b>what</b> to do, but not <b>how</b> to do it. <br></br>

Once 6:00 hits all trains are activated and express routes are implemented. For example, the late night A-train might go to certain stops, but it skips over them in the day. In the day you wouldn't be able to sit on the A-train the entire time to get to every stop, you would have to get off and make a transfer to the C-train to get to all the local stops. On that note this is why the goal is to determine a set of paths and not just a single path. The only weight the algorithm understands is the time between stations, it doesn't understand that train switching is expensive. Everytime you get off a train you have to wait for the next one to arrive which adds to the overall time. Therefore, the algorithm can only return a set of potential options that a human would need to then filter through.

<h2> Solving the Problem </h2>

Like any route-inspection style problem this problem is about decision making, specifically what are you going to do at junctions (nodes with degree greater than 2). Let's look at the simple network below to understand the idea. The challenge is to stop at every station and you plan on starting at A. You can basically ignore station B and C because to get from A to D you don't have a choice but to stop at those stations. It's when you get to station D a choice has to be made. Do you go to F first, come back to D, and then go to E or do you go to E first come back and go to F. Taking the ABC<b>DFD</b>E route will cost you 28 compared to the 20 it will cost you taking the ABC<b>DED</b>F. What makes the Subway Challenge so tricky is that taking the optimal DED route requires you to get off the blue train you were already on and wait for the red train and then again on the way back for the blue train. Whereas the less optimal route requires you to get off the blue train for the red train once. If the wait time at D for a train is over 8 then the suboptimal DFD route becomes the optimal route. 

<figure style='text-align:center'>
    <img src="./Images/Network-Simple.png" style='max-width:50%'/>
    <figcaption style='text-align:center'> Figure 1. Simple Graph to Understand the Problem </figcaption>
</figure>

Let's resist the idea of ignoring stations B and C begin. As stated previously, at those stations you have no choices and on your way to D you will pass them anyway. If that is the case, then you can reduce the network by removing them (<i>Figure 2</i>) and change the how you the challenge is viewed. The nodes in the network are all the stations, the nodes are only the stations where you have to make a decision. Or in other words, the stations that allow you to transfer to a different train. With that, what you are trying to do is travel every edge <b>at least once</b>. By doing that you will automatically stop at every station. This modification officially turns our problem into a <b>Chinese Postman Problem</b> (CPP). In simplest terms, the Chinese postman problem aims is to find a path that visits every edge of a graph. 

<figure style='text-align:center'>
    <img src="./Images/Network-Simple-Reduced.png" style='max-width:50%'/>
    <figcaption style='text-align:center'> Figure 2. The Simple Graph Reduced </figcaption>
</figure>

<h2> Building on a Prepackaged Solution </h2>

In 2017 Andrew Brooks was tackling a similar problem and he solved it using the NetworkX 2.0 library <a href="https://www.datacamp.com/community/tutorials/networkx-python-graph-tutorial"> []</a> . Thankfully, he packaged his solution into the <a href="https://github.com/brooksandrew/postman_problems">postman_problems</a> package. With this package you can plug in your own network and solve the CPP problem. Unfortunately, the Subway Challenge isn't a typical CPP problem. The postman always wants to return to his vehicle, so the CPP finds a path that always returns to the starting point. The Subway Challenge has no such requirement, the sole requirement is to travel to all the edges at least once. Andrew's postman_package finds solves the CPP as is, therefore plugging in the subway network wouldn't work because it would return us to where we started which is a suboptimal solution. However, Andrew did do all of the heavy lifting and with a little bit of network theory, the NetworkX 2.5 update, and some tweaks to his package, we could build on his work to solve the problem. 

<p style='text-align:center'> <b> X.1 The Graph Theory Behind Andrew's Solution</b> </p>
In graph theory a path is a sequence of vertices with the property that each vertex in the sequence is adjacent to the vertex next to it. A circuit is a path that begins and ends at the same vertex. If we think of it in terms of racing, a path is a street race. You don't necessarily end up at the same point where you started (although you could) but all the streets are connected to each other. A circuit is a nascar race where you always end where you start. An Euler circuit not only has to meet the conditions of a regular circuit (starting and ending at the same place) it has the added conditions that you have to use <b>every</b> edge of the graph AND you can only use each edge <b>once</b>. This is where the where the issues begin; there is a theorem that states: <br></br>
<p style='text-align:center'> A connected graph $G$ has an Euler Circuit if and only if every vertex has even degree <p>
In Andrew's problem there are many nodes that have odd degrees which means there isn't an euler circuit. So his solution was to turn odd-degree nodes even by adding artificial edges to odd-degree nodes. Then when all the nodes are even he uses NetworkX to find the euler circuit. 

<p style='text-align:right'> <b>X.1.A Understanding The Steps</b> </p>
First he loads in the edge list and then creates the graph from the edge list. The graph that we will use to understand his methodology is shown below. The graph is an weighted undirected graph where the orange nodes are the odd degree nodes. 

<figure style='text-align:center'>
    <img src="./Images/Network-Start.png" style='max-width:20%'/>
    <figcaption style='text-align:center'> Figure 1. The Graph for the Follow-Along </figcaption>
</figure>

<b>The theorem states that only graphs where all nodes are even degree qualify to have euler circuits so his first efforts are to make all the nodes even.</b> The first thing that he does is create a separate graph where the odd nodes are artifically paired together. The artificial edges are shown in red and the weights on them represent the <b>fastest</b> time to get from the nodes using <b>actual</b> paths (<i>In this separate graph the even degree nodes and their connections don't exist</i>). 

<figure style='text-align:center'>
    <img src="./Images/Network-Start-Odd-Augment.png" style='max-width:20%'/>
    <figcaption style='text-align:center'> Figure 2. Odd Nodes Complete Graph via Artificial Paths </figcaption>
</figure>

All the paths in red aren't neccessary, because to turn an odd degree node even you only need to add a single path. This is where the idea of a matching comes into play, specifically a minimum weight matching. Listed below are three different ways to think about matchings:
<ol>
    <li> A matching is a subset of edges in which no node occurs more than once. </li>
    <li> A matching is a graph where all the nodes have a degree of 1 or 0. </li>
    <li> A matching is a subgraph of a graph where there are no edges adjacent to each other </li>
</ol>

The weight of a matching is the sum of the weights of its edges and the cardinality of a matching is the number of matched edges. What we are looking for is a matching that has <b>maximum cardinality</b> but <b>minimum weight</b>. Our three choices of matchings are shown below and the one that is ultimately chosen is the middle one that has the minimum weight of 11. 

<figure style='text-align:center'>
    <img src="./Images/Network-Matchings.png" style='max-width:50%'/>
    <figcaption style='text-align:center'> Figure 3. Matchings with Maximum Cardinality</figcaption>
</figure>

Those two edges are added to the original graph and now all the nodes are even. Remember, the AE edge doesn't exist so when the algorithm says to follow the AE path, what happens in actuality is you have to go from node A to B to E or reverse. <b>The point of the previous steps is that we don't have a choice but to reuse a path already traveled so we need to find which path/s require the least amount of work to redo.</b> The final augmented graph is shown below. From here the NetworkX 2.0 package is used to return the circuit. 

<figure style='text-align:center'>
    <img src="./Images/Network-Final-Augment.png" style='max-width:20%'/>
    <figcaption style='text-align:center'> Figure 3. Matchings with Maximum Cardinality</figcaption>
</figure>

<p style='text-align:center'> <b> X.2 The Graph Theory To Build on Andrew's Solution</b> </p>
Andrew's solution solves for the Euler circuit which isn't what we are looking for, we are looking for an Euler Path. An Euler Path is a path that has the added conditions that you have to use <b>every</b> edge of the graph <b>exactly once</b>. The difference between the two is that in an Euler Path we don't have to go back to where we began. With an Euler Path there is a different theorem that will guide our work:<br></br>

<p style='text-align:center'> If a graph $G$ has an Euler Path, then it must have <b>exactly two odd vertices.</b></p>

Essentially, odd-degree nodes are dead-ends. There is going to come a time when you reach the node and there are no more edges for you to use to leave the node. In an Euler Path these dead-ends serve as the starting and ending nodes. In Andrew's problem he turned all the odd-degree nodes even, in our problem we have to turn all but two of those nodes even. Therefore, all we had to do was find the set of odd nodes, remove two of them, and then follow Andrew's steps. Those two conserved nodes will be the starting point and the ending point. The question then became, which two odd-degree nodes do we conserve. Choosing where to start and where to end is part of the difficulty of the Subway Challenge.

The only start and endpoint known is Matthew Ahn's pair and we don't know that that pair is ideal. Therefore every odd-degree node could be a potential start node or a potential end node and thus we have $\dbinom{O}{2}$ configurations to check, where $O$ is the number of odd-degree nodes. For every configuration we will remove those two odd-degree nodes and then follow the same steps to come up with the path/s that require the least amount of effort to doubleback. 